In [2]:
pip install undetected-chromedriver

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47130 sha256=44f2d05505d4d1151f6794d9fc33eeed4ed98348cc0171bc0081e987643df5c2
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\c4\f1\aa\9de6cf276210554d91e9c0526864563e850a428c5e76da4914
Successfully built undetected-chromedriver
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


## oyuncu özeli

In [1]:
import re
import requests

from bs4 import BeautifulSoup

In [2]:
# Scrape a player page
url = "https://www.transfermarkt.us/erling-haaland/profil/spieler/418560"
player_id = url.split('/')[-1]

In [3]:
# Get headers from https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

In [4]:
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
player_name = soup.select_one('h1[class="data-header__headline-wrapper"]').text.split('\n')[-1].strip()
player_number = soup.select_one('span[class="data-header__shirt-number"]').text.strip().replace('#', '')

In [6]:
# Let's use regex to find the player's contract expiry date
player_contract_expiry = re.search("Contract expires: (.*)", soup.text).group(1)

# Now let's add some more complex regex patterns to find the player's birthplace
player_birthplace = re.search("Place of birth:.*?([A-z].*?) ", soup.text, re.DOTALL).group(1).strip()

# We can reuse both of those ideas to find other player information but just replacing the name

# We might have to switch a couple of things though in the regex pattern like accounting for two words
player_agent = re.search("Agent:.*?([A-z].*?)\n", soup.text, re.DOTALL).group(1).strip()

# We can also use regex to find the player's height which has numbers a comma and letters
player_height = re.search("Height:.*?([0-9].*?)\n", soup.text, re.DOTALL).group(1).strip()

In [10]:
# Ok so now that we've got some basic regex patterns, let's move on to using api endpoints
# curlconverter.com
api_endpoints = [
    f"marketValueDevelopment/graph/{player_id}",
    f"transferHistory/list/{player_id}",
    f"player/{player_id}/performance"
]

for endpoint in api_endpoints:
    response = requests.get(
        'https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/418560' + endpoint,
        headers=headers
    ).json()

In [9]:
player_height

'1,95 m'

## -----------------

## Api kullanarak Oyuncunun tüm transfermarkt değerlerinin çekilmesi 

In [11]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [12]:
import requests

player_id = "418560"
url = f"https://www.transfermarkt.us/erling-haaland/profil/spieler/{player_id}"
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}
# Oyuncu adının cek
response_html = requests.get(url, headers=headers)
soup = BeautifulSoup(response_html.content, "html.parser")
player_name = soup.select_one('h1.data-header__headline-wrapper').text.split('\n')[-1].strip()

# Market value verisinin cekilmesi
response = requests.get(
    f'https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}',
    headers=headers,
)
print(response.status_code)
print(response.text)  

200
{"list":[{"x":1482015600000,"y":200000,"mw":"200 bin \u20ac","datum_mw":"18 Ara 2016","verein":"Bryne FK","age":"16","wappen":"https:\/\/tmssl.akamaized.net\/\/images\/wappen\/profil\/1057.png?lm=1480871779"},{"x":1513983600000,"y":300000,"mw":"300 bin \u20ac","datum_mw":"23 Ara 2017","verein":"Molde FK","age":"17","wappen":"https:\/\/tmssl.akamaized.net\/\/images\/wappen\/profil\/687.png?lm=1409159512"},{"x":1536530400000,"y":2000000,"mw":"2.00 mil. \u20ac","datum_mw":"10 Eyl 2018","verein":"Molde FK","age":"18","wappen":""},{"x":1546124400000,"y":5000000,"mw":"5.00 mil. \u20ac","datum_mw":"30 Ara 2018","verein":"Molde FK","age":"18","wappen":""},{"x":1559512800000,"y":5000000,"mw":"5.00 mil. \u20ac","datum_mw":"3 Haz 2019","verein":"Red Bull Salzburg","age":"18","wappen":"https:\/\/tmssl.akamaized.net\/\/images\/wappen\/profil\/409_1557781653.png?lm=1557781653"},{"x":1567461600000,"y":12000000,"mw":"12.00 mil. \u20ac","datum_mw":"3 Eyl 2019","verein":"Red Bull Salzburg","age":"19

In [13]:
data = response.json()

In [14]:
# Doğru anahtar: "list"
df = pd.DataFrame(data["list"])

In [15]:
df

,x,y,mw,datum_mw,verein,age,wappen
0,1482015600000,200000,200 bin €,18 Ara 2016,Bryne FK,16,https://tmssl.akamaized.net//images/wappen/pro...
1,1513983600000,300000,300 bin €,23 Ara 2017,Molde FK,17,https://tmssl.akamaized.net//images/wappen/pro...
2,1536530400000,2000000,2.00 mil. €,10 Eyl 2018,Molde FK,18,
3,1546124400000,5000000,5.00 mil. €,30 Ara 2018,Molde FK,18,
4,1559512800000,5000000,5.00 mil. €,3 Haz 2019,Red Bull Salzburg,18,https://tmssl.akamaized.net//images/wappen/pro...
5,1567461600000,12000000,12.00 mil. €,3 Eyl 2019,Red Bull Salzburg,19,
6,1573081200000,30000000,30.00 mil. €,7 Kas 2019,Red Bull Salzburg,19,
7,1576450800000,45000000,45.00 mil. €,16 Ara 2019,Red Bull Salzburg,19,
8,1581375600000,60000000,60.00 mil. €,11 Şub 2020,Borussia Dortmund,19,https://tmssl.akamaized.net//images/wappen/pro...
9,1583881200000,80000000,80.00 mil. €,11 Mar 2020,Borussia Dortmund,19,


In [16]:
# (Opsiyonel) Tarih sütununu dönüştür
if "x" in df.columns:
    df["date"] = pd.to_datetime(df["x"], unit="ms")

print(df.head())

               x        y           mw     datum_mw             verein age  \
0  1482015600000   200000    200 bin €  18 Ara 2016           Bryne FK  16   
1  1513983600000   300000    300 bin €  23 Ara 2017           Molde FK  17   
2  1536530400000  2000000  2.00 mil. €  10 Eyl 2018           Molde FK  18   
3  1546124400000  5000000  5.00 mil. €  30 Ara 2018           Molde FK  18   
4  1559512800000  5000000  5.00 mil. €   3 Haz 2019  Red Bull Salzburg  18   

                                              wappen                date  
0  https://tmssl.akamaized.net//images/wappen/pro... 2016-12-17 23:00:00  
1  https://tmssl.akamaized.net//images/wappen/pro... 2017-12-22 23:00:00  
2                                                    2018-09-09 22:00:00  
3                                                    2018-12-29 23:00:00  
4  https://tmssl.akamaized.net//images/wappen/pro... 2019-06-02 22:00:00  


In [17]:
# Oyuncu adını ekle
df["player_name"] = player_name

print(df.head())

               x        y           mw     datum_mw             verein age  \
0  1482015600000   200000    200 bin €  18 Ara 2016           Bryne FK  16   
1  1513983600000   300000    300 bin €  23 Ara 2017           Molde FK  17   
2  1536530400000  2000000  2.00 mil. €  10 Eyl 2018           Molde FK  18   
3  1546124400000  5000000  5.00 mil. €  30 Ara 2018           Molde FK  18   
4  1559512800000  5000000  5.00 mil. €   3 Haz 2019  Red Bull Salzburg  18   

                                              wappen                date  \
0  https://tmssl.akamaized.net//images/wappen/pro... 2016-12-17 23:00:00   
1  https://tmssl.akamaized.net//images/wappen/pro... 2017-12-22 23:00:00   
2                                                    2018-09-09 22:00:00   
3                                                    2018-12-29 23:00:00   
4  https://tmssl.akamaized.net//images/wappen/pro... 2019-06-02 22:00:00   

      player_name  
0  Erling Haaland  
1  Erling Haaland  
2  Erling Haal

In [18]:
df

,x,y,mw,datum_mw,verein,age,wappen,date,player_name
0,1482015600000,200000,200 bin €,18 Ara 2016,Bryne FK,16,https://tmssl.akamaized.net//images/wappen/pro...,2016-12-17 23:00:00,Erling Haaland
1,1513983600000,300000,300 bin €,23 Ara 2017,Molde FK,17,https://tmssl.akamaized.net//images/wappen/pro...,2017-12-22 23:00:00,Erling Haaland
2,1536530400000,2000000,2.00 mil. €,10 Eyl 2018,Molde FK,18,,2018-09-09 22:00:00,Erling Haaland
3,1546124400000,5000000,5.00 mil. €,30 Ara 2018,Molde FK,18,,2018-12-29 23:00:00,Erling Haaland
4,1559512800000,5000000,5.00 mil. €,3 Haz 2019,Red Bull Salzburg,18,https://tmssl.akamaized.net//images/wappen/pro...,2019-06-02 22:00:00,Erling Haaland
5,1567461600000,12000000,12.00 mil. €,3 Eyl 2019,Red Bull Salzburg,19,,2019-09-02 22:00:00,Erling Haaland
6,1573081200000,30000000,30.00 mil. €,7 Kas 2019,Red Bull Salzburg,19,,2019-11-06 23:00:00,Erling Haaland
7,1576450800000,45000000,45.00 mil. €,16 Ara 2019,Red Bull Salzburg,19,,2019-12-15 23:00:00,Erling Haaland
8,1581375600000,60000000,60.00 mil. €,11 Şub 2020,Borussia Dortmund,19,https://tmssl.akamaized.net//images/wappen/pro...,2020-02-10 23:00:00,Erling Haaland
9,1583881200000,80000000,80.00 mil. €,11 Mar 2020,Borussia Dortmund,19,,2020-03-10 23:00:00,Erling Haaland


## Lig bazlı oyuncuların 2023-2024-2025 Transfermarkt değerlerinin çekilmesi

## bundesliga için

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urlencode

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

positions = [
    {"pos": "Torwart", "detailpos": "1", "name": "Goalkeeper"},
    {"pos": "Abwehr", "detailpos": "3", "name": "Centre-Back"},
    {"pos": "Abwehr", "detailpos": "5", "name": "Left-Back"},
    {"pos": "Abwehr", "detailpos": "4", "name": "Right-Back"},
    {"pos": "Mittelfeld", "detailpos": "6", "name": "Defensive Midfield"},
    {"pos": "Mittelfeld", "detailpos": "7", "name": "Central Midfield"},
    {"pos": "Mittelfeld", "detailpos": "10", "name": "Attacking Midfield"},
    {"pos": "Mittelfeld", "detailpos": "9", "name": "Left Midfield"},
    {"pos": "Mittelfeld", "detailpos": "8", "name": "Right Midfield"},
    {"pos": "Sturm", "detailpos": "13", "name": "Second Striker"},
    {"pos": "Sturm", "detailpos": "14", "name": "Centre-Forward"},
    {"pos": "Sturm", "detailpos": "12", "name": "Right Winger"},
    {"pos": "Sturm", "detailpos": "11", "name": "Left Winger"},
]

In [3]:
base_url = "https://www.transfermarkt.com/bundesliga/marktwerte/wettbewerb/L1/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)
df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

              name               club age market_value    position player_id  \
0     Gregor Kobel  Borussia Dortmund          €40.00m  Goalkeeper    257814   
1     Noah Atubolu        SC Freiburg          €18.00m  Goalkeeper    526845   
2    Kamil Grabara      VfL Wolfsburg          €14.00m  Goalkeeper    346551   
3      Jonas Urbig      Bayern Munich          €12.00m  Goalkeeper    607720   
4  Alexander Nübel      VfB Stuttgart          €12.00m  Goalkeeper    195778   

  market_value_2023  Market_Value_2023 market_value_2024  Market_Value_2024  \
0      40.00 mil. €         40000000.0      40.00 mil. €         40000000.0   
1       6.50 mil. €          6500000.0      10.00 mil. €         10000000.0   
2      11.00 mil. €         11000000.0      14.00 mil. €         14000000.0   
3       3.00 mil. €          3000000.0       3.00 mil. €          3000000.0   
4      10.00 mil. €         10000000.0      12.00 mil. €         12000000.0   

  market_value_2025  Market_Value_2025  
0  

In [4]:
df

,name,club,age,market_value,position,player_id,market_value_2023,Market_Value_2023,market_value_2024,Market_Value_2024,market_value_2025,Market_Value_2025
0,Gregor Kobel,Borussia Dortmund,,€40.00m,Goalkeeper,257814,40.00 mil. €,40000000.0,40.00 mil. €,40000000.0,40.00 mil. €,40000000.0
1,Noah Atubolu,SC Freiburg,,€18.00m,Goalkeeper,526845,6.50 mil. €,6500000.0,10.00 mil. €,10000000.0,18.00 mil. €,18000000.0
2,Kamil Grabara,VfL Wolfsburg,,€14.00m,Goalkeeper,346551,11.00 mil. €,11000000.0,14.00 mil. €,14000000.0,14.00 mil. €,14000000.0
3,Jonas Urbig,Bayern Munich,,€12.00m,Goalkeeper,607720,3.00 mil. €,3000000.0,3.00 mil. €,3000000.0,12.00 mil. €,12000000.0
4,Alexander Nübel,VfB Stuttgart,,€12.00m,Goalkeeper,195778,10.00 mil. €,10000000.0,12.00 mil. €,12000000.0,12.00 mil. €,12000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
519,Yan Diomande,RB Leipzig,,€1.50m,Left Winger,1390649,None,NaN,None,NaN,1.50 mil. €,1500000.0
520,Yvandro Borges Sanches,Borussia Mönchengladbach,,€500k,Left Winger,586659,1.00 mil. €,1000000.0,500 bin €,500000.0,500 bin €,500000.0
521,David Preu,1.FC Union Berlin,,€400k,Left Winger,745075,50 bin €,50000.0,25 bin €,25000.0,400 bin €,400000.0
522,Levin Öztunali,Hamburger SV,,€300k,Left Winger,168543,900 bin €,900000.0,400 bin €,400000.0,300 bin €,300000.0


In [5]:
df.to_excel('almanya_market_values.xlsx')

## jupiler pro league

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urlencode

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

positions = [
    {"pos": "Torwart", "detailpos": "1", "name": "Goalkeeper"},
    {"pos": "Abwehr", "detailpos": "3", "name": "Centre-Back"},
    {"pos": "Abwehr", "detailpos": "5", "name": "Left-Back"},
    {"pos": "Abwehr", "detailpos": "4", "name": "Right-Back"},
    {"pos": "Mittelfeld", "detailpos": "6", "name": "Defensive Midfield"},
    {"pos": "Mittelfeld", "detailpos": "7", "name": "Central Midfield"},
    {"pos": "Mittelfeld", "detailpos": "10", "name": "Attacking Midfield"},
    {"pos": "Mittelfeld", "detailpos": "9", "name": "Left Midfield"},
    {"pos": "Mittelfeld", "detailpos": "8", "name": "Right Midfield"},
    {"pos": "Sturm", "detailpos": "13", "name": "Second Striker"},
    {"pos": "Sturm", "detailpos": "14", "name": "Centre-Forward"},
    {"pos": "Sturm", "detailpos": "12", "name": "Right Winger"},
    {"pos": "Sturm", "detailpos": "11", "name": "Left Winger"},
]

In [3]:
base_url = "https://www.transfermarkt.com/jupiler-pro-league/marktwerte/wettbewerb/BE1/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)

df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

# Sadece integer market value sütunlarını uzun formata çeviriyoruz
value_cols = [f"Market_Value_{year}" for year in years]
id_cols = ["name", "club", "position", "player_id"]

df_long = pd.melt(
    df,
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="year",
    value_name="yearly_market_value"  # <-- burada farklı bir isim kullandık
)

df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.dropna(subset=["yearly_market_value"])

print(df_long.head())

             name                  club age market_value    position  \
0   Senne Lammens      Royal Antwerp FC           €9.00m  Goalkeeper   
1  Kjell Scherpen  Union Saint-Gilloise           €5.00m  Goalkeeper   
2     Tobe Leysen   Oud-Heverlee Leuven           €4.00m  Goalkeeper   
3  Matthieu Epolo        Standard Liège           €4.00m  Goalkeeper   
4      Leo Kokubo     Sint-Truidense VV           €3.00m  Goalkeeper   

  player_id market_value_2023  Market_Value_2023 market_value_2024  \
0    503883         700 bin €           700000.0       4.50 mil. €   
1    463527       3.00 mil. €          3000000.0       4.00 mil. €   
2    580434       1.00 mil. €          1000000.0       3.50 mil. €   
3    680218         500 bin €           500000.0       3.00 mil. €   
4    501737         750 bin €           750000.0       2.50 mil. €   

   Market_Value_2024 market_value_2025  Market_Value_2025  
0          4500000.0       9.00 mil. €          9000000.0  
1          4000000.0      

In [5]:
df_long

,name,club,position,player_id,year,yearly_market_value
0,Senne Lammens,Royal Antwerp FC,Goalkeeper,503883,2023,700000.0
1,Kjell Scherpen,Union Saint-Gilloise,Goalkeeper,463527,2023,3000000.0
2,Tobe Leysen,Oud-Heverlee Leuven,Goalkeeper,580434,2023,1000000.0
3,Matthieu Epolo,Standard Liège,Goalkeeper,680218,2023,500000.0
4,Leo Kokubo,Sint-Truidense VV,Goalkeeper,501737,2023,750000.0
...,...,...,...,...,...,...
1279,Ibrahim Kanaté,RSC Anderlecht,Left Winger,1132628,2025,500000.0
1280,Théo Epailly,RAAL La Louvière,Left Winger,395514,2025,400000.0
1281,Abdoulaye Traoré,Zulte Waregem,Left Winger,1063856,2025,400000.0
1282,Kenny Nagera,RAAL La Louvière,Left Winger,618416,2025,350000.0


In [6]:
df_long.to_excel('belcika_market_values.xlsx')

## bundesliga

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urlencode

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

positions = [
    {"pos": "Torwart", "detailpos": "1", "name": "Goalkeeper"},
    {"pos": "Abwehr", "detailpos": "3", "name": "Centre-Back"},
    {"pos": "Abwehr", "detailpos": "5", "name": "Left-Back"},
    {"pos": "Abwehr", "detailpos": "4", "name": "Right-Back"},
    {"pos": "Mittelfeld", "detailpos": "6", "name": "Defensive Midfield"},
    {"pos": "Mittelfeld", "detailpos": "7", "name": "Central Midfield"},
    {"pos": "Mittelfeld", "detailpos": "10", "name": "Attacking Midfield"},
    {"pos": "Mittelfeld", "detailpos": "9", "name": "Left Midfield"},
    {"pos": "Mittelfeld", "detailpos": "8", "name": "Right Midfield"},
    {"pos": "Sturm", "detailpos": "13", "name": "Second Striker"},
    {"pos": "Sturm", "detailpos": "14", "name": "Centre-Forward"},
    {"pos": "Sturm", "detailpos": "12", "name": "Right Winger"},
    {"pos": "Sturm", "detailpos": "11", "name": "Left Winger"},
]

In [4]:
base_url = "https://www.transfermarkt.com/bundesliga/marktwerte/wettbewerb/L1/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)

df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

# Sadece integer market value sütunlarını uzun formata çeviriyoruz
value_cols = [f"Market_Value_{year}" for year in years]
id_cols = ["name", "club", "position", "player_id"]

df_long = pd.melt(
    df,
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="year",
    value_name="yearly_market_value"  # <-- burada farklı bir isim kullandık
)

df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.dropna(subset=["yearly_market_value"])

print(df_long.head())

              name               club age market_value    position player_id  \
0     Gregor Kobel  Borussia Dortmund          €40.00m  Goalkeeper    257814   
1     Noah Atubolu        SC Freiburg          €18.00m  Goalkeeper    526845   
2    Kamil Grabara      VfL Wolfsburg          €14.00m  Goalkeeper    346551   
3      Jonas Urbig      Bayern Munich          €12.00m  Goalkeeper    607720   
4  Alexander Nübel      VfB Stuttgart          €12.00m  Goalkeeper    195778   

  market_value_2023  Market_Value_2023 market_value_2024  Market_Value_2024  \
0      40.00 mil. €         40000000.0      40.00 mil. €         40000000.0   
1       6.50 mil. €          6500000.0      10.00 mil. €         10000000.0   
2      11.00 mil. €         11000000.0      14.00 mil. €         14000000.0   
3       3.00 mil. €          3000000.0       3.00 mil. €          3000000.0   
4      10.00 mil. €         10000000.0      12.00 mil. €         12000000.0   

  market_value_2025  Market_Value_2025  
0  

In [5]:
df_long

,name,club,position,player_id,year,yearly_market_value
0,Gregor Kobel,Borussia Dortmund,Goalkeeper,257814,2023,40000000.0
1,Noah Atubolu,SC Freiburg,Goalkeeper,526845,2023,6500000.0
2,Kamil Grabara,VfL Wolfsburg,Goalkeeper,346551,2023,11000000.0
3,Jonas Urbig,Bayern Munich,Goalkeeper,607720,2023,3000000.0
4,Alexander Nübel,VfB Stuttgart,Goalkeeper,195778,2023,10000000.0
...,...,...,...,...,...,...
1564,Yan Diomande,RB Leipzig,Left Winger,1390649,2025,1500000.0
1565,Yvandro Borges Sanches,Borussia Mönchengladbach,Left Winger,586659,2025,500000.0
1566,David Preu,1.FC Union Berlin,Left Winger,745075,2025,400000.0
1567,Levin Öztunali,Hamburger SV,Left Winger,168543,2025,300000.0


In [6]:
df_long.to_excel('almanya_market_values.xlsx')

## Hollanda

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urlencode

In [8]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

positions = [
    {"pos": "Torwart", "detailpos": "1", "name": "Goalkeeper"},
    {"pos": "Abwehr", "detailpos": "3", "name": "Centre-Back"},
    {"pos": "Abwehr", "detailpos": "5", "name": "Left-Back"},
    {"pos": "Abwehr", "detailpos": "4", "name": "Right-Back"},
    {"pos": "Mittelfeld", "detailpos": "6", "name": "Defensive Midfield"},
    {"pos": "Mittelfeld", "detailpos": "7", "name": "Central Midfield"},
    {"pos": "Mittelfeld", "detailpos": "10", "name": "Attacking Midfield"},
    {"pos": "Mittelfeld", "detailpos": "9", "name": "Left Midfield"},
    {"pos": "Mittelfeld", "detailpos": "8", "name": "Right Midfield"},
    {"pos": "Sturm", "detailpos": "13", "name": "Second Striker"},
    {"pos": "Sturm", "detailpos": "14", "name": "Centre-Forward"},
    {"pos": "Sturm", "detailpos": "12", "name": "Right Winger"},
    {"pos": "Sturm", "detailpos": "11", "name": "Left Winger"},
]

In [9]:
base_url = "https://www.transfermarkt.com/eredivisie/marktwerte/wettbewerb/NL1/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)

df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

# Sadece integer market value sütunlarını uzun formata çeviriyoruz
value_cols = [f"Market_Value_{year}" for year in years]
id_cols = ["name", "club", "position", "player_id"]

df_long = pd.melt(
    df,
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="year",
    value_name="yearly_market_value"  # <-- burada farklı bir isim kullandık
)

df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.dropna(subset=["yearly_market_value"])

print(df_long.head())

                      name                 club age market_value    position  \
0      Timon Wellenreuther  Feyenoord Rotterdam           €6.00m  Goalkeeper   
1  Rome Jayden Owusu-Oduro           AZ Alkmaar           €6.00m  Goalkeeper   
2            Justin Bijlow  Feyenoord Rotterdam           €5.00m  Goalkeeper   
3              Matej Kovar        PSV Eindhoven           €5.00m  Goalkeeper   
4          Vitezslav Jaros       Ajax Amsterdam           €4.00m  Goalkeeper   

  player_id market_value_2023  Market_Value_2023 market_value_2024  \
0    196829       3.00 mil. €          3000000.0       6.00 mil. €   
1    704772         500 bin €           500000.0       3.00 mil. €   
2    315121      15.00 mil. €         15000000.0       6.50 mil. €   
3    550829       5.00 mil. €          5000000.0       7.00 mil. €   
4    486604         250 bin €           250000.0       5.00 mil. €   

   Market_Value_2024 market_value_2025  Market_Value_2025  
0          6000000.0       6.00 mil. €

In [10]:
df_long.to_excel('hollanda_market_values.xlsx')

## Premier League

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urlencode

In [12]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

positions = [
    {"pos": "Torwart", "detailpos": "1", "name": "Goalkeeper"},
    {"pos": "Abwehr", "detailpos": "3", "name": "Centre-Back"},
    {"pos": "Abwehr", "detailpos": "5", "name": "Left-Back"},
    {"pos": "Abwehr", "detailpos": "4", "name": "Right-Back"},
    {"pos": "Mittelfeld", "detailpos": "6", "name": "Defensive Midfield"},
    {"pos": "Mittelfeld", "detailpos": "7", "name": "Central Midfield"},
    {"pos": "Mittelfeld", "detailpos": "10", "name": "Attacking Midfield"},
    {"pos": "Mittelfeld", "detailpos": "9", "name": "Left Midfield"},
    {"pos": "Mittelfeld", "detailpos": "8", "name": "Right Midfield"},
    {"pos": "Sturm", "detailpos": "13", "name": "Second Striker"},
    {"pos": "Sturm", "detailpos": "14", "name": "Centre-Forward"},
    {"pos": "Sturm", "detailpos": "12", "name": "Right Winger"},
    {"pos": "Sturm", "detailpos": "11", "name": "Left Winger"},
]

In [13]:
base_url = "https://www.transfermarkt.com/premier-league/marktwerte/wettbewerb/GB1/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)

df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

# Sadece integer market value sütunlarını uzun formata çeviriyoruz
value_cols = [f"Market_Value_{year}" for year in years]
id_cols = ["name", "club", "position", "player_id"]

df_long = pd.melt(
    df,
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="year",
    value_name="yearly_market_value"  # <-- burada farklı bir isim kullandık
)

df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.dropna(subset=["yearly_market_value"])

print(df_long.head())

                   name                    club age market_value    position  \
0            David Raya              Arsenal FC          €40.00m  Goalkeeper   
1     Guglielmo Vicario       Tottenham Hotspur          €32.00m  Goalkeeper   
2       Bart Verbruggen  Brighton & Hove Albion          €30.00m  Goalkeeper   
3  Giorgi Mamardashvili            Liverpool FC          €30.00m  Goalkeeper   
4           André Onana       Manchester United          €25.00m  Goalkeeper   

  player_id market_value_2023  Market_Value_2023 market_value_2024  \
0    262749      35.00 mil. €         35000000.0      40.00 mil. €   
1    286047      30.00 mil. €         30000000.0      35.00 mil. €   
2    565093      15.00 mil. €         15000000.0      25.00 mil. €   
3    502676      30.00 mil. €         30000000.0      40.00 mil. €   
4    234509      40.00 mil. €         40000000.0      35.00 mil. €   

   Market_Value_2024 market_value_2025  Market_Value_2025  
0         40000000.0      40.00 mil. €

In [14]:
df_long.to_excel('ingiltere_market_values.xlsx')

## La Liga

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urlencode

In [16]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

positions = [
    {"pos": "Torwart", "detailpos": "1", "name": "Goalkeeper"},
    {"pos": "Abwehr", "detailpos": "3", "name": "Centre-Back"},
    {"pos": "Abwehr", "detailpos": "5", "name": "Left-Back"},
    {"pos": "Abwehr", "detailpos": "4", "name": "Right-Back"},
    {"pos": "Mittelfeld", "detailpos": "6", "name": "Defensive Midfield"},
    {"pos": "Mittelfeld", "detailpos": "7", "name": "Central Midfield"},
    {"pos": "Mittelfeld", "detailpos": "10", "name": "Attacking Midfield"},
    {"pos": "Mittelfeld", "detailpos": "9", "name": "Left Midfield"},
    {"pos": "Mittelfeld", "detailpos": "8", "name": "Right Midfield"},
    {"pos": "Sturm", "detailpos": "13", "name": "Second Striker"},
    {"pos": "Sturm", "detailpos": "14", "name": "Centre-Forward"},
    {"pos": "Sturm", "detailpos": "12", "name": "Right Winger"},
    {"pos": "Sturm", "detailpos": "11", "name": "Left Winger"},
]

In [17]:
base_url = "https://www.transfermarkt.com/laliga/marktwerte/wettbewerb/ES1/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)

df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

# Sadece integer market value sütunlarını uzun formata çeviriyoruz
value_cols = [f"Market_Value_{year}" for year in years]
id_cols = ["name", "club", "position", "player_id"]

df_long = pd.melt(
    df,
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="year",
    value_name="yearly_market_value"  # <-- burada farklı bir isim kullandık
)

df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.dropna(subset=["yearly_market_value"])

print(df_long.head())

               name                club age market_value    position  \
0        Unai Simón     Athletic Bilbao          €28.00m  Goalkeeper   
1       Joan García        FC Barcelona          €25.00m  Goalkeeper   
2         Jan Oblak  Atlético de Madrid          €20.00m  Goalkeeper   
3  Thibaut Courtois         Real Madrid          €20.00m  Goalkeeper   
4       Álex Remiro       Real Sociedad          €18.00m  Goalkeeper   

  player_id market_value_2023  Market_Value_2023 market_value_2024  \
0    262396      25.00 mil. €         25000000.0      28.00 mil. €   
1    561613         400 bin €           400000.0      10.00 mil. €   
2    121483      35.00 mil. €         35000000.0      25.00 mil. €   
3    108390      35.00 mil. €         35000000.0      25.00 mil. €   
4    212862      25.00 mil. €         25000000.0      25.00 mil. €   

   Market_Value_2024 market_value_2025  Market_Value_2025  
0         28000000.0      28.00 mil. €         28000000.0  
1         10000000.0      

In [18]:
df_long.to_excel('ispanya_market_values.xlsx')

## Seria A

In [19]:
base_url = "https://www.transfermarkt.com/seria/marktwerte/wettbewerb/IT1/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)

df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

# Sadece integer market value sütunlarını uzun formata çeviriyoruz
value_cols = [f"Market_Value_{year}" for year in years]
id_cols = ["name", "club", "position", "player_id"]

df_long = pd.melt(
    df,
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="year",
    value_name="yearly_market_value"  # <-- burada farklı bir isim kullandık
)

df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.dropna(subset=["yearly_market_value"])

print(df_long.head())

                name               club age market_value    position  \
0       Mike Maignan           AC Milan          €25.00m  Goalkeeper   
1        Mile Svilar            AS Roma          €25.00m  Goalkeeper   
2  Marco Carnesecchi        Atalanta BC          €25.00m  Goalkeeper   
3        Zion Suzuki  Parma Calcio 1913          €20.00m  Goalkeeper   
4         Alex Meret         SSC Napoli          €18.00m  Goalkeeper   

  player_id market_value_2023  Market_Value_2023 market_value_2024  \
0    182906      45.00 mil. €         45000000.0      35.00 mil. €   
1    338670       1.50 mil. €          1500000.0      14.00 mil. €   
2    459763      13.00 mil. €         13000000.0      20.00 mil. €   
3    432298       1.00 mil. €          1000000.0       9.00 mil. €   
4    240414      15.00 mil. €         15000000.0      12.00 mil. €   

   Market_Value_2024 market_value_2025  Market_Value_2025  
0         35000000.0      25.00 mil. €         25000000.0  
1         14000000.0      

In [20]:
df_long.to_excel('italya_market_values.xlsx')

## Portugal

In [21]:
base_url = "https://www.transfermarkt.com/liga-nos/marktwerte/wettbewerb/PO1/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)

df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

# Sadece integer market value sütunlarını uzun formata çeviriyoruz
value_cols = [f"Market_Value_{year}" for year in years]
id_cols = ["name", "club", "position", "player_id"]

df_long = pd.melt(
    df,
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="year",
    value_name="yearly_market_value"  # <-- burada farklı bir isim kullandık
)

df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.dropna(subset=["yearly_market_value"])

print(df_long.head())

              name            club age market_value    position player_id  \
0      Diogo Costa        FC Porto          €40.00m  Goalkeeper    357153   
1  Anatoliy Trubin      SL Benfica          €28.00m  Goalkeeper    537844   
2        Rui Silva     Sporting CP           €7.00m  Goalkeeper    234811   
3   Lukas Hornicek        SC Braga           €6.00m  Goalkeeper    629773   
4  Gabriel Batista  CD Santa Clara           €4.00m  Goalkeeper    401535   

  market_value_2023  Market_Value_2023 market_value_2024  Market_Value_2024  \
0      45.00 mil. €         45000000.0      40.00 mil. €         40000000.0   
1      25.00 mil. €         25000000.0      25.00 mil. €         25000000.0   
2      15.00 mil. €         15000000.0       5.00 mil. €          5000000.0   
3       2.00 mil. €          2000000.0       2.00 mil. €          2000000.0   
4         500 bin €           500000.0       1.50 mil. €          1500000.0   

  market_value_2025  Market_Value_2025  
0      40.00 mil. €  

In [22]:
df_long.to_excel('portekiz_market_values.xlsx')

## Ligue 1

In [23]:
base_url = "https://www.transfermarkt.com/ligue-1/marktwerte/wettbewerb/FR1/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)

df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

# Sadece integer market value sütunlarını uzun formata çeviriyoruz
value_cols = [f"Market_Value_{year}" for year in years]
id_cols = ["name", "club", "position", "player_id"]

df_long = pd.melt(
    df,
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="year",
    value_name="yearly_market_value"  # <-- burada farklı bir isim kullandık
)

df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.dropna(subset=["yearly_market_value"])

print(df_long.head())

                   name                  club age market_value    position  \
0  Gianluigi Donnarumma   Paris Saint-Germain          €40.00m  Goalkeeper   
1       Lucas Chevalier            LOSC Lille          €30.00m  Goalkeeper   
2        Matvey Safonov   Paris Saint-Germain          €20.00m  Goalkeeper   
3      Guillaume Restes           FC Toulouse          €20.00m  Goalkeeper   
4          Mike Penders  RC Strasbourg Alsace          €12.00m  Goalkeeper   

  player_id market_value_2023  Market_Value_2023 market_value_2024  \
0    315858      40.00 mil. €         40000000.0      35.00 mil. €   
1    463600      18.00 mil. €         18000000.0      30.00 mil. €   
2    318470      18.00 mil. €         18000000.0      22.00 mil. €   
3    912902      10.00 mil. €         10000000.0      18.00 mil. €   
4    834397         800 bin €           800000.0       5.00 mil. €   

   Market_Value_2024 market_value_2025  Market_Value_2025  
0         35000000.0      40.00 mil. €         400

In [24]:
df_long.to_excel('fransa_market_values.xlsx')

## Championship

In [25]:
base_url = "https://www.transfermarkt.com/championship/marktwerte/wettbewerb/GB2/plus//galerie/0"
all_players = []
seen_players = set()

for pos_info in positions:
    for page in range(1, 5):
        query_params = {
            "pos": pos_info["pos"],
            "detailpos": pos_info["detailpos"],
            "altersklasse": "alle",
            "land_id": "0"
        }
        if page == 1:
            url = f"{base_url}?{urlencode(query_params)}"
        else:
            url = f"{base_url}/page/{page}?{urlencode(query_params)}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"class": "items"})
        if not table:
            continue
        rows = table.find_all("tr", {"class": ["odd", "even"]})
        for row in rows:
            try:
                name_cell = row.find("td", {"class": "hauptlink"})
                name = name_cell.text.strip()
                profile_link = name_cell.find("a")["href"]
                player_id = profile_link.split("/")[-1]
                zentriert_cells = row.find_all("td", class_="zentriert")
                club = ""
                for cell in zentriert_cells:
                    club_link = cell.find("a", title=True)
                    if club_link:
                        club = club_link["title"]
                        break
                if not club:
                    continue
                key = f"{name}|{club}"
                if key in seen_players:
                    continue
                age = row.find_all("td")[2].text.strip()
                market_value = row.find_all("td")[-1].text.strip()
                all_players.append({
                    "name": name,
                    "club": club,
                    "age": age,
                    "market_value": market_value,
                    "position": pos_info["name"],
                    "player_id": player_id,
                })
                seen_players.add(key)
            except Exception as e:
                continue
        time.sleep(1)

df = pd.DataFrame(all_players)
def convert_market_value(val):
    if not isinstance(val, str):
        return None
    val = val.replace("€", "").replace(",", "").strip()
    if "mil" in val:
        num = val.split("mil")[0].strip()
        try:
            return round(float(num) * 1_000_000)
        except:
            return None
    elif "bin" in val:
        num = val.split("bin")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    elif "k" in val:
        num = val.split("k")[0].strip()
        try:
            return round(float(num) * 1_000)
        except:
            return None
    else:
        return None
def get_market_values_by_year(player_id, years=[2023, 2024, 2025]):
    url = f"https://www.transfermarkt.com.tr/ceapi/marketValueDevelopment/graph/{player_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {year: None for year in years}
    data = response.json()
    result = {year: None for year in years}
    for entry in data.get("list", []):
        date = pd.to_datetime(entry["x"], unit="ms")
        year = date.year
        if year in years:
            if (result[year] is None) or (date > result[year]["date"]):
                result[year] = {
                    "market_value": entry["y"],
                    "market_value_str": entry["mw"],
                    "date": date,
                }
    return {year: (result[year]["market_value_str"] if result[year] else None) for year in years}

years = [2023, 2024, 2025]
for idx, row in df.iterrows():
    player_id = row["player_id"]
    values = get_market_values_by_year(player_id, years)
    for year in years:
        # String değer
        df.at[idx, f"market_value_{year}"] = values[year]
        # Integer değer
        df.at[idx, f"Market_Value_{year}"] = convert_market_value(values[year])
    time.sleep(0.5)  # Transfermarkt'ı yormamak için bekleme

print(df.head())

# Sadece integer market value sütunlarını uzun formata çeviriyoruz
value_cols = [f"Market_Value_{year}" for year in years]
id_cols = ["name", "club", "position", "player_id"]

df_long = pd.melt(
    df,
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="year",
    value_name="yearly_market_value"  # <-- burada farklı bir isim kullandık
)

df_long["year"] = df_long["year"].str.extract(r"(\d{4})").astype(int)
df_long = df_long.dropna(subset=["yearly_market_value"])

print(df_long.head())

               name              club age market_value    position player_id  \
0    Aaron Ramsdale    Southampton FC          €16.00m  Goalkeeper    427568   
1    Mads Hermansen    Leicester City          €15.00m  Goalkeeper    400536   
2      Gavin Bazunu    Southampton FC           €9.00m  Goalkeeper    585550   
3    Michael Cooper  Sheffield United           €9.00m  Goalkeeper    533078   
4  Viktor Johansson        Stoke City           €8.00m  Goalkeeper    273742   

  market_value_2023  Market_Value_2023 market_value_2024  Market_Value_2024  \
0      28.00 mil. €         28000000.0      20.00 mil. €         20000000.0   
1       9.00 mil. €          9000000.0      12.00 mil. €         12000000.0   
2      15.00 mil. €         15000000.0      14.00 mil. €         14000000.0   
3       1.00 mil. €          1000000.0       7.00 mil. €          7000000.0   
4       3.00 mil. €          3000000.0       5.00 mil. €          5000000.0   

  market_value_2025  Market_Value_2025  
0  

In [26]:
df_long.to_excel('ingiltere_iki_market_values.xlsx')

## Gruplama

In [1]:
import pandas as pd
import os

In [3]:
# Excel dosyalarının bulunduğu klasör yolu
klasor_yolu = "C:/Users/user/Desktop/PROJELER/FutbolcuOneriRevize/transfermarkt/scraping"  
cikis_dosyasi = "birlesik_transfermarkt.xlsx"

In [4]:
# Tüm .xlsx dosyalarını listele
dosyalar = [f for f in os.listdir(klasor_yolu) if f.endswith(".xlsx")]

In [5]:
# Boş bir liste
tum_veriler = []

In [6]:
# Her dosyayı oku ve listeye ekle
for dosya in dosyalar:
    dosya_yolu = os.path.join(klasor_yolu, dosya)
    df = pd.read_excel(dosya_yolu)
    tum_veriler.append(df)

In [7]:
# Tüm verileri alt alta birleştir
birlesik_df = pd.concat(tum_veriler, ignore_index=True)

In [8]:
#Tek bir sayfa olarak yaz
birlesik_df.to_excel(cikis_dosyasi, index=False)

print("Birleştirme tamamlandı:", cikis_dosyasi)

Birleştirme tamamlandı: birlesik_transfermarkt.xlsx
